# Case Study: Zeeguu/API
- Backend of a web application that supports [free reading in foreign languages](https://zeeguu.org)
- Open source [repository on GH](https://github.com/zeeguu/API/)

## Table of conentents
1. [Basic Data Gathering](#basic-data-gathering)
    1. [Extract dependencies](#extract-dependencies)
    2. [Visualize](#Visualize)
2. [Abstraction](#Abstraction)
3. [Evolution](#Evolution)
4. [Dynamic Analysis](#dynamic-analysis)



## Basic Data Gathering

- extracting basic dependencies between python modules
- every .py file is called a module in Python
- direct relationship between file name and module name
  - file: `./zeeguu_core/model/user.py` <==>
  - module: `zeeguu_core.model.User`


In [1]:
# Credit: https://colab.research.google.com/drive/1oe_TV7936Zmmzbbgq8rzqFpxYPX7SQHP#scrollTo=Njkjj4fzUV2E
# Installing Required Dependencies
import sys
sys.version
!{sys.executable} -m pip install gitpython
!{sys.executable} -m pip install pyvis

In [2]:
# Adopted from: https://colab.research.google.com/drive/1oe_TV7936Zmmzbbgq8rzqFpxYPX7SQHP#scrollTo=Njkjj4fzUV2E
import os
from git import Repo

# Current Working Directory
cwd = os.getcwd()
print(cwd)

# Code location
CODE_ROOT_FOLDER=f"/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/"

# Clone the repository
if not os.path.exists(CODE_ROOT_FOLDER):
  Repo.clone_from("https://github.com/zeeguu/api", CODE_ROOT_FOLDER)



/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction


In [3]:
# Count absolute lines of code and number of files 
!cd {CODE_ROOT_FOLDER} && git ls-files | grep '\.py$' | xargs wc -l | grep total
!cd {CODE_ROOT_FOLDER} && git ls-files | grep "\.py$" | wc -l

   21206 total
     278


In [4]:
# helpers
def file_path(file_name):
    return f"{CODE_ROOT_FOLDER}{file_name}"


def module_name_from_file_path(full_path):
    """
    ../core/model/user.py -> zeeguu.core.model.user
    """
    file_name = full_path[len(CODE_ROOT_FOLDER):]
    file_name = file_name.replace("/__init__.py","")
    file_name = file_name.replace("/",".")
    file_name = file_name.replace(".py","")
    return file_name

File_Name = "zeeguu/core/model/user.py"
print(file_path(File_Name))
assert file_path(File_Name) == "/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/user.py"
assert module_name_from_file_path(file_path(File_Name)) == "zeeguu.core.model.user"


def module_name_from_rel_path(full_path):

    # e.g. ../core/model/user.py -> zeeguu.core.model.user

    file_name = full_path.replace("/__init__.py","")
    file_name = file_name.replace("/",".")
    file_name = file_name.replace(".py","")
    return file_name

assert ("tools.migrations.teacher_dashboard_migration_1.upgrade" == module_name_from_rel_path("tools/migrations/teacher_dashboard_migration_1/upgrade.py"))
assert ("zeeguu.api") == module_name_from_rel_path("zeeguu/api/__init__.py")


  

/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/user.py


### AST based parsing

In [102]:
# Import class

class Import:
    def __init__(self, module:str, total_calls:int=0):
        self.module = module
        self.function_calls = defaultdict(int)
        self.distinct_calls = set()
        self.total_calls = total_calls

    def __str__(self):
        return (f"Import: module {self.module}\n"
                f"Function calls {[f'{k} : {v}' for k,v in self.function_calls.items()]}\n"
                f"Distinct calls {self.distinct_calls}\n"
                f"Total calls: {self.total_calls}\n")



In [103]:
import ast
from collections import defaultdict


def parse_imports(file):
    with open(file) as f:
        tree = ast.parse(f.read(), filename=file)
    
    imports = set() # all imported modules
    imports_v : defaultdict[str, Import] = defaultdict(lambda : Import("None") )
    import_total_function_calls = defaultdict(lambda : defaultdict(int)) # total calls to functions in imported modules
    import_distinct_function_calls = defaultdict(lambda: set()) # unique calls to functions in imported modules
    import_total = defaultdict(int) # total calls to imported modules
    function_to_module = {} # alias to module mapping
    function_to_module_function = {} # alias to module mapping
    
    try:
        for node in ast.walk(tree):
            match node:
                case ast.Import(names=names ):
                    for alias in names:
                        imports.add(alias.name)
                        imports_v[alias.name] = Import(alias.name)
                
                case ast.ImportFrom(module=module, names=names):
                    # What if module == .?
                    if module == None:
                        #imports.add(".") Internal imports. Disregard for now
                        continue
                    
                    imports.add(module)
                    imports_v[module] = Import(module)
                    
                    for alias in names:
                        function_to_module_function[alias.name] = module + "." + alias.name
                        function_to_module[alias.name] = module

                # The imported function is assigned to a variable
                # case ast.Assign(_):
                #     print(f"Assign: {node}")
                case ast.Assign(targets=[ast.Name(id=id)], value=ast.Call(func=ast.Name (id=name), args=args, keywords=keywords)):
                    if name in function_to_module: 
                        function_to_module[id] = function_to_module[name]
                        function_to_module_function[id] = function_to_module_function[name]
                        if function_to_module[id] in imports_v.keys():
                            imports_v[function_to_module[id]].function_calls[f"{function_to_module_function[id]}"] += 1
                            imports_v[function_to_module[id]].total_calls += 1
                            imports_v[function_to_module[id]].distinct_calls.add(f"{function_to_module_function[id]}")
                        
                        
                    
                case ast.Call(func=ast.Attribute(value=ast.Name(id=id), attr=attr), args=args, keywords=keywords):
                    if id in function_to_module:
                        if function_to_module[id] in imports_v.keys():
                            imports_v[function_to_module[id]].function_calls[f"{function_to_module_function[id]}.{attr}"] += 1
                            imports_v[function_to_module[id]].total_calls += 1
                            imports_v[function_to_module[id]].distinct_calls.add(f"{function_to_module_function[id]}.{attr}")
                            
                        
    except Exception as e:
        print(f"Error in {file}: {e}")
        print(f"Module: {module}")
        print(f"Alias: {alias.name}")
                    

    return imports, imports_v


print(parse_imports(file_path('zeeguu/cl/__init__.py'))[1]["zeeguu.api.app"])
print("Stop")
assert parse_imports(file_path('zeeguu/core/model/unique_code.py'))[0] == {'datetime', 'zeeguu.core', 'zeeguu.core.model', 'sqlalchemy', 'random'}

imports = parse_imports(file_path('zeeguu/core/model/unique_code.py'))
print(f"Imports: {imports[0]}")
print(f"Import_v: {imports[1]}")
print(f"Import_total: {imports[1]['zeeguu.core.model']}")


Import: module zeeguu.api.app
Function calls ['zeeguu.api.app.create_app : 1', 'zeeguu.api.app.create_app.app_context : 1']
Distinct calls {'zeeguu.api.app.create_app', 'zeeguu.api.app.create_app.app_context'}
Total calls: 2

Stop
Imports: {'random', 'zeeguu.core.model', 'sqlalchemy', 'zeeguu.core', 'datetime'}
Import_v: defaultdict(<function parse_imports.<locals>.<lambda> at 0x1047599e0>, {'datetime': <__main__.Import object at 0x11a3ecc20>, 'random': <__main__.Import object at 0x11a3ed730>, 'zeeguu.core': <__main__.Import object at 0x11a3ef110>, 'sqlalchemy': <__main__.Import object at 0x11a3eddc0>, 'zeeguu.core.model': <__main__.Import object at 0x11a3eedb0>})
Import_total: Import: module zeeguu.core.model
Function calls ['zeeguu.core.model.db.Column : 4', 'zeeguu.core.model.db.String : 2']
Distinct calls {'zeeguu.core.model.db.Column', 'zeeguu.core.model.db.String'}
Total calls: 6



### Extract dependencies and visalize
To do that we iterate over all the python files with the help of the Path.rglob function from pathlib
And we create a network with the help of the networkx package.Visualize

In [7]:
# TODO: Add dependency data on hover
# TODO: Add coloration depending on node level
import pyvis.network as Network
import matplotlib.pyplot as plt
from pathlib import Path
import networkx as nx

def dependencies_digraph(code_root_folder):
    files = Path(code_root_folder).rglob("*.py")

    G = nx.DiGraph()

    for file in files:
        file_path = str(file)

        source_module = module_name_from_file_path(file_path)

        if source_module not in G.nodes:
            G.add_node(source_module)

        imports, _ = parse_imports(file_path)
        for target_module in imports:

            G.add_edge(source_module, target_module)
            # print(module_name + "=>" + each + ".")

    return G

# a function to draw a graph
def draw_graph_plt(G, size, **args):
    plt.figure(figsize=size)
    nx.draw(G, **args)
    plt.show()
    
def draw_graph_pyvis(G, size, output_file, **args):
    h, w = size
    # Adjaency list
    neighbors = G.adj
    # Add neighbors on hover
    for node in G.nodes:
        G.nodes[node]["title"] = " Imports:\n" + "\n".join(neighbors[node])
        G.nodes[node]["value"] = len(neighbors[node])
        G.nodes[node]["font"] = {"size": 32, "color": "white"}
    
    g = Network.Network(height=h, width=w,notebook=True, cdn_resources='in_line', directed=True,  **args) 
    for n in g.nodes:
        n["size"] = 100
        n["font"]={"size": 100}
    g.show_buttons(filter_=['physics'])
    g.from_nx(G)
    g.barnes_hut()
    #g.toggle_physics(True)
    
    g.show(output_file)

def draw_graph_pyvis_example():
    G = nx.complete_graph(5)
    draw_graph_pyvis(G, ("1000px",  "100%"), "example.html", bgcolor="#222222", font_color="white")   
    
    


In [8]:
# Looking at the directed graph
DG = dependencies_digraph(CODE_ROOT_FOLDER)
draw_graph_pyvis(DG, (1000,1000), "draw_all.html", bgcolor="#222222", font_color="white")
# draw_graph_plt(DG, (10,10), with_labels=True, node_size=1000, font_size=10)

/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/starred_article.py:71: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/article.py:28: SyntaxWarning: invalid escape sequence '\s'
  MULTIPLE_NEWLINES = re.compile("\n\s*\n")
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/language/strategies/word_rank_difficulty_estimator.py:63: SyntaxWarning: invalid escape sequence '\d'
  words = [w for w in words if re.search("\d", w) == None]


draw_all.html


## Abstraction
What do we have now:
- System: zeeguu/api
- Source View: Modules & Dependencies
- Entities: .py files in the project
- Relationships: import statements between .py files

Plan: Abstraction methods
1. Folder hierarchy
2. Aggregate dependencies using metrics. (Sum of calls)
    - Total count of explicit low-level dependencies
    - Number of distinct explicit low-level dependencies
    - Network analysis to detect rank packages: Note (It should not be that hard, the networkx package supports various methods of network analysis, e.g. centrality, HITS, pagerank.)
3. Create different level graphs and pass them to OpenAI vision model

### Filter relevant modules

In [9]:

def relevant_module(module_name):
    """
    Define relevant modules
    """
    if "test" in module_name:
        return False
    if module_name.startswith("zeeguu"):
        return True


    return False

def dependencies_digraph_filtered(code_root_folder):
    files = Path(code_root_folder).rglob("*.py")

    G = nx.DiGraph()

    for file in files:
        file_path = str(file)

        source_module = module_name_from_file_path(file_path)
        if not relevant_module(source_module):
          continue

        if source_module not in G.nodes:
            G.add_node(source_module)
            
        imports, imports_v= parse_imports(file_path)
        for target_module in imports_v.keys():

            if relevant_module(target_module):
                
                import_object = imports_v[target_module]
                G.add_edge(source_module, target_module, value=import_object.total_calls,
                           title=f"Total calls: {import_object.total_calls}\n"
                                 f"Distinct calls: {len(import_object.distinct_calls)}\n"
                                 f"Functions: \n{"\n".join(import_object.distinct_calls)}")


    return G

# Looking at the directed graph
DG = dependencies_digraph_filtered(CODE_ROOT_FOLDER)
draw_graph_pyvis(DG, ("1000px",  "100%"), "draw_all_filtered.html", bgcolor="#222222", font_color="white")
print(DG)

draw_all_filtered.html
DiGraph with 182 nodes and 422 edges


/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/starred_article.py:71: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/article.py:28: SyntaxWarning: invalid escape sequence '\s'
  MULTIPLE_NEWLINES = re.compile("\n\s*\n")
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/language/strategies/word_rank_difficulty_estimator.py:63: SyntaxWarning: invalid escape sequence '\d'
  words = [w for w in words if re.search("\d", w) == None]


### Basic Abstraction Using Hierarchical Module Structure & Naming Conventions

- abstracting the imports between the modules along the module hierarchy
- also taking into account naming conventions to filter out external modules

In [127]:
def top_level_package(module_name, depth=1):
    """Extract parent of module at depth"""
    components = module_name.split(".")
    return ".".join(components[:depth])

assert (top_level_package("zeeguu.core.model.util") == "zeeguu")
assert (top_level_package("zeeguu.core.model.util", 2) == "zeeguu.core")

def merge_imports(import1 : Import, import2 : Import) -> Import:
    """
    Merge two imports into a new import
    :param import1: Import to merge into
    :param import2: Import to merge from
    :return: import 1 with merged values
    """
    merged : Import = Import(import1.module)
    merged.total_calls = import1.total_calls + import2.total_calls
    merged.distinct_calls = import1.distinct_calls.union(import2.distinct_calls)
    merged.function_calls = import1.function_calls.copy()
    
    for k,v in import2.function_calls.items():
        merged.function_calls[k] += v
    
    return merged
    
assert merge_imports(Import("zeeguu.core", 10), Import("zeeguu.core.model", 20)).total_calls == 30
assert merge_imports(Import("zeeguu.core", 10), Import("zeeguu.core.model", 20)).module == "zeeguu.core"

def merge_into_top_level(import_map : defaultdict[str, defaultdict[str, Import]], source_module, imports_v, depth=1):
    top = top_level_package(source_module, depth)
    if not top in import_map.keys():
        import_map[top] = defaultdict(lambda: Import("None"))
                
    assert top_level_package(source_module, depth) == top_level_package(top, depth)
    
    for target in imports_v.keys():
        if relevant_module(target):
            #print(f"Source: {source_module} Top: {top} Target: {target}")
            import_map[top][target].module = target
            import_map[top][target] = merge_imports(import_map[top][target], imports_v[target])
            
    return import_map

assert merge_into_top_level( defaultdict(), "zeeguu.core.model.util", {"zeeguu.core.model": Import("zeeguu.core.model", 10)}, 2)["zeeguu.core"]["zeeguu.core.model"].total_calls == 10
assert merge_into_top_level(defaultdict(), "zeeguu.core.model.util", {"zeeguu.core.model": Import("zeeguu.core.model", 10)}, 2)["zeeguu.core"]["zeeguu.core.model"].module == "zeeguu.core.model"

In [132]:
def dependencies_digraph_filtered_v2(code_root_folder, depth=2) -> (nx.DiGraph, defaultdict[str, defaultdict[str, Import]]):
    files = Path(code_root_folder).rglob("*.py")
    import_map : defaultdict[str, defaultdict[str, Import]] =  defaultdict(lambda : defaultdict(lambda: Import("None")))

    G = nx.DiGraph()
    
    for file in files:
        file_path = str(file)

        source_module = module_name_from_file_path(file_path)
        if not relevant_module(source_module):
            continue
        
        imports, imports_v = parse_imports(file_path)
        top = top_level_package(source_module, depth)
        
        if len(source_module.split(".")) > len(top.split(".")):
            # zeeeguu.core.model > zeeguu.core
            # Add edge values to top level package
            merge_into_top_level(import_map, source_module, imports_v, depth)
        
        import_map[source_module] = imports_v
        
        if source_module not in G.nodes:
            G.add_node(source_module)
        
        for target_module in imports_v.keys():

            if relevant_module(target_module):
                
                import_object = imports_v[target_module]
                G.add_edge(source_module, target_module, value=import_object.total_calls,
                           title=f"Total calls: {import_object.total_calls}\n"
                                 f"Distinct calls: {len(import_object.distinct_calls)}\n"
                                 f"Functions: \n{"\n".join(import_object.distinct_calls)}")
    
    
    return G, import_map

def abstracted_to_top_level(G, import_map, depth=1):
    aG = nx.DiGraph()
    abstracted_import_map : defaultdict[str, defaultdict[str, Import]] =  defaultdict(lambda : defaultdict(lambda: Import("None")))
    
    for edge in G.edges():
        src = top_level_package(edge[0], depth)
        dst = top_level_package(edge[1], depth)
    
        if src != dst:
            all_imports = import_map[edge].copy()
            relavent_imports = []
            count = 0
            
            # Copy imports to top level package
            for k_module,_import in all_imports.items():
                if top_level_package(k_module, depth) == dst: # if top_level(zeeguu.core.model) == zeeguu.core
                    
                    abstracted_import_map[src][k_module].module = k_module
                    abstracted_import_map[src][k_module] = merge_imports(all_imports[k_module], _import)
                    
                    for k_func,v in _import.function_calls.items():
                        relavent_imports.append((k_func, v))
                        count += v
                        
            #print(relavent_imports)
            aG.add_edge(src, dst, value= count, title=f"Total calls: {count}\n"
                                     f"Distinct calls: {len(relavent_imports)}\n"
                                     f"Functions: \n{"\n".join([f'{k} : {v}' for k,v in sorted(relavent_imports, key=lambda x: x[1], reverse=True)])}")

    return aG, abstracted_import_map


depth = 2
G, import_map = dependencies_digraph_filtered_v2(CODE_ROOT_FOLDER, depth)
AG, a_import_map = abstracted_to_top_level(G, import_map, depth)
print(G)
print(f"Import map: {import_map["zeeguu.core"].keys()}")
print(f"Abstracted map {a_import_map["zeeguu.core"].keys()}")
print(AG)


DiGraph with 182 nodes and 422 edges
Import map: dict_keys(['zeeguu.core.constants', 'zeeguu.logging', 'zeeguu.core.model', 'zeeguu.core.emailer.user_activity', 'zeeguu.core', 'zeeguu.core.util.encoding', 'zeeguu.core.util.hash', 'zeeguu.core.util.time', 'zeeguu.core.model.article', 'zeeguu.core.model.difficulty_lingo_rank', 'zeeguu.core.elastic.settings', 'zeeguu.core.elastic.elastic_query_builder', 'zeeguu.core.util.timer_logging_decorator', 'zeeguu.core.ml_models', 'zeeguu.core.content_cleaning', 'zeeguu.core.content_retriever.crawler_exceptions', 'zeeguu', 'zeeguu.core.elastic.indexing', 'zeeguu.core.content_quality.quality_filter', 'zeeguu.core.emailer.zeeguu_mailer', 'zeeguu.core.content_retriever', 'zeeguu.core.language.difficulty_estimator_strategy', 'zeeguu.core.language.strategies.default_difficulty_estimator', 'zeeguu.core.language.strategies.flesch_kincaid_difficulty_estimator', 'zeeguu.core.word_stats', 'zeeguu.core.model.language', 'zeeguu.core.language.difficulty_estimat

/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/starred_article.py:71: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/article.py:28: SyntaxWarning: invalid escape sequence '\s'
  MULTIPLE_NEWLINES = re.compile("\n\s*\n")
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/language/strategies/word_rank_difficulty_estimator.py:63: SyntaxWarning: invalid escape sequence '\d'
  words = [w for w in words if re.search("\d", w) == None]


In [27]:
def draw_graph_pyvis_v2(G, size, output_file, **args):
    h, w = size
    # Adjaency list
    neighbors = G.adj
    # Add neighbors on hover
    for node in G.nodes:
        G.nodes[node]["title"] = " Imports:\n" + "\n".join(neighbors[node])
        G.nodes[node]["group"] = top_level_package(node, 2)
    
    # Scaling the size of the nodes by 5*degree
    scale = 3 # Scaling the size of the nodes by 10*degree
    degrees = dict(G.degree())
    degrees.update((x, scale*y) for x, y in degrees.items())
    nx.set_node_attributes(G, degrees, "size")
        
    g = Network.Network(height=h, width=w,notebook=True, cdn_resources='in_line', directed=True,  **args) 
    g.show_buttons()
    g.from_nx(G)
    g.barnes_hut()
    g.show(output_file)
    
DG = dependencies_digraph_filtered_v2(CODE_ROOT_FOLDER, 3)
ADG = abstracted_to_top_level(DG, 3)
draw_graph_pyvis_v2(ADG, ("800px","100%"), "draw_all_filtered_abstracted2.html", bgcolor="#222222", font_color="white")

/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/starred_article.py:71: SyntaxWarning: invalid escape sequence '\ '
  """
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/model/article.py:28: SyntaxWarning: invalid escape sequence '\s'
  MULTIPLE_NEWLINES = re.compile("\n\s*\n")
/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api/zeeguu/core/language/strategies/word_rank_difficulty_estimator.py:63: SyntaxWarning: invalid escape sequence '\d'
  words = [w for w in words if re.search("\d", w) == None]


draw_all_filtered_abstracted2.html


### Abstraction with metrics

### Abstraction with network analysis

## Evolution
Plan:
1.  Churn Find hot code -- Most changed/imporant regions

2. Extract multiple complementary module views from your case study system
3. Ensure that your layouts are readable - limit the number of nodes in a view, use a different layout in networkx, or use a different library than networkx
4. Augment each of the previously obtained module views by mapping the above-computed churn metric on the color of a given node

In [ ]:
!{sys.executable} -m pip install pydriller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.6 MB/s eta 0:00:00


In [ ]:
from pydriller import Repository
REPO_DIR = 'https://github.com/zeeguu/api'

In [ ]:
# for PyDriller to work we need to change directory to our local clone of the repo
%cd {CODE_ROOT_FOLDER}

/Users/andreaskongstad/Developer/PycharmProjects/architectural-reconstruction/data/zeeguu-api


In [ ]:
from collections import defaultdict
from pydriller import ModificationType

all_commits = list(Repository(REPO_DIR).traverse_commits())

def print_out_commit_details(commits):
    """ Usage: print_out_commit_details(all_commits[0:1])"""
    for commit in commits:
        print(commit)
        for each in commit.modified_files:
            print(f"{commit.author.name} {each.change_type} {each.filename}\n -{each.old_path}\n -{each.new_path}")


def commit_counts(all_commits):
    commit_counts = defaultdict(int)

    for commit in all_commits:
        for file in commit.modified_files:
            commit_counts[file.new_path] += 1

    return commit_counts


def commit_counts_better(all_commits):
    commit_counts = {}
    for commit in all_commits:
        for modification in commit.modified_files:

            new_path = modification.new_path
            old_path = modification.old_path

            try:
                if modification.change_type == ModificationType.RENAME:
                    commit_counts[new_path]=commit_counts.get(old_path,0)+1
                    commit_counts.pop(old_path)

                elif modification.change_type == ModificationType.DELETE:
                    commit_counts.pop(old_path, '')

                elif modification.change_type == ModificationType.ADD:
                    commit_counts[new_path] = 1

                else: # modification to existing file
                        commit_counts [old_path] += 1
            except Exception as e:
                print("something went wrong with: " + str(modification))
                pass
        return commit_counts
        
        

# sort by number of commits in decreasing order
commit_counts = commit_counts_better(all_commits)
sorted_commits = sorted(commit_counts.items(), key=lambda x: x[1], reverse=True)[:42]
# discussion: What is ("None", 103) ?

In [ ]:
def package_activity():
    package_activity = defaultdict(int)

    for path, count in commit_counts.items():
        if ".py" in str(path):
            l2_module = top_level_package(module_name_from_rel_path(path), 2)
            package_activity[l2_module] += count

    return package_activity

package_activity = package_activity()
sorted_sizes = sorted(package_activity.items(), key=lambda x: x[1], reverse=True)

In [ ]:
plt.figure(figsize=(7,7))
nx.draw_networkx(ADG, with_labels=True, node_size = sizes, node_color='r')
plt.show()


NameError: name 'nx' is not defined

<Figure size 700x700 with 0 Axes>

## Dynamic Analysis
Not as relavent for project.
Plan: Dont know if i will do this yet.

In [ ]:
import inspect

def methods_in_class(cls):
    """ Returns all the methods in a class """
    return [
		(name, object) 
		for (name, object) 
			in cls.__dict__.items() 
		if hasattr(object, '__call__')]
    
def log_decorator( function ):
    """ A decorator that logs the function on call """
    def decorated( *args, **kwargs ):
        print (f'I have been called: {function}')
        return function( *args,**kwargs )
    return decorated

def decorate_methods( cls, decorator ):
    """ Decorates all the methods in a class with a log_decorator"""
    methods = methods_in_class(cls)
    for name, method in methods:
	    setattr( cls, name, decorator ( method ))
    return cls


def caller(): 
	callee()

def callee():
    """ Prints the name of the calling function"""
    print(inspect.stack()[1].function)

caller()



In [ ]:
# Decoreate the user class:
from zeeguu.core.model import User
decorate_methods(User, log_decorator)

u= User.find_by_id(534)
u.bookmark_count()

# to see even further one can instrument also third party libraries!
from sqlalchemy.orm.query import Query
decorate_methods(Query, log_decorator)
